In [1]:
import numpy as np
import jax
import jax.numpy as jnp
import jimgw
import lal

In [68]:
import numpy as np

import os

os.environ['XLA_PYTHON_CLIENT_PREALLOCATE']='false'
import optax
import corner
import matplotlib.pyplot as plt
import corner
import jax                                           
#jax.config.update('jax_platform_name', 'cpu')

import jax.numpy as jnp
from functools import partial

import distrax
import haiku as hk      #neural network library for JAX 
from typing import Any, Iterator, Mapping, Optional, Sequence, Tuple
Array = jnp.ndarray
PRNGKey = Array
OptState = Any


In [2]:
from interferometer import Interferometer
from scipy.interpolate import interp1d
import numpy as np
import jax.numpy as jnp
import gw
from matplotlib import pyplot as pl
import corner
import pickle
%matplotlib inline

In [29]:
f_min = 0. #minimum frequency cut-off
T = 1.  #data segment duration (8 seconds)
#Fs = 2048 #samplingfrequency  (Hz)
F2 = 20.
#t_start = 1126259446. #strain data segment start time 
t_start=0.
# since signals are generated in frequency domain this time is only used for calculating time dealys and hence is set to t_coalsecence

#initialise the detectors
H1_d = Interferometer('H1','O3',f_min,T,Fs,t_start,20011997)
L1_d = Interferometer('L1','O3',f_min,T,Fs,t_start,27071994)

In [30]:
def simulate_fd_sine_gaussian_waveform(A, t0, f0, tau, times, fmin, df):   #without phase for now
    
    t = times
    
    hpt = A*jnp.exp(-(t-t0)**2/tau**2)*jnp.cos(2*jnp.pi*f0*t) #time domain plus polarisation
    hct = A*jnp.exp(-(t-t0)**2/tau**2)*jnp.sin(2*jnp.pi*f0*t) #time domain cross polarisation

    hp = jnp.fft.rfft(hpt) #frequency domain plus polarisation
    hc = jnp.fft.rfft(hct) #frequency domain cross polarisation

    return hp, hc

In [199]:
A_big=jnp.array([[10.,5.]])

In [209]:
A_big.T.shape

(2, 1)

In [195]:
A_big.shape

AttributeError: 'list' object has no attribute 'shape'

In [215]:
jnp.matmul(A_big.T, t)

Array([[0.00000000e+00, 4.88281250e-03, 9.76562500e-03, ...,
        9.98535156e+00, 9.99023438e+00, 9.99511719e+00],
       [0.00000000e+00, 2.44140625e-03, 4.88281250e-03, ...,
        4.99267578e+00, 4.99511719e+00, 4.99755859e+00]], dtype=float64)

In [210]:
simulate_fd_sine_gaussian_waveform(A_big.T, t0, f0, tau, t, f_min, H1_d.df)

(Array([[-9.66092332e+01+0.00000000e+00j, -1.17429113e+02-3.41671136e-14j,
         -2.19427697e+02+8.49320614e-14j, ...,
         -3.59274232e-03-4.29656311e-14j, -3.59271696e-03+5.10980147e-14j,
         -3.59270850e-03+0.00000000e+00j],
        [-4.83046166e+01+0.00000000e+00j, -5.87145565e+01-1.70835568e-14j,
         -1.09713849e+02+4.24660307e-14j, ...,
         -1.79637116e-03-2.14828155e-14j, -1.79635848e-03+2.55490074e-14j,
         -1.79635425e-03+0.00000000e+00j]], dtype=complex128),
 Array([[ 1.17498371e-13+0.00000000e+00j,  2.67424971e-14+5.58682660e+01j,
          1.07247544e-13+1.76779850e+02j, ...,
         -7.77156117e-15+1.35267356e-07j,  1.23373534e-14+6.76330245e-08j,
          1.84664772e-14+0.00000000e+00j],
        [ 5.87491855e-14+0.00000000e+00j,  1.33712486e-14+2.79341330e+01j,
          5.36237721e-14+8.83899252e+01j, ...,
         -3.88578059e-15+6.76336782e-08j,  6.16867668e-15+3.38165123e-08j,
          9.23323859e-15+0.00000000e+00j]], dtype=complex128))

In [170]:
t=H1_d.times #time array for the time domain signal
A=10
t0=0.5
f0=4.
tau=2./f0
ra=1.5
dec=0.5
psi=1.0


simulate_fd_sine_gaussian_waveform(A, t0, f0, tau, t, f_min, H1_d.df)

#simulate signal 
hp, hc = simulate_fd_sine_gaussian_waveform(A,t0,f0,tau,t,0.,H1_d.df)
hct = jnp.fft.irfft(hc)
pl.plot(t,hct)
print(hp,hc)

TypeError: unsupported operand type(s) for *: 'list' and 'Array'

In [55]:
from curses import KEY_REPLACE
import lalsimulation as lalsim 
import numpy as np
import jax.numpy as jnp
import jax

# from ripple.waveforms.IMRPhenomD import gen_IMRPhenomD_polar
from ripple import ms_to_Mc_eta
from ripple.waveforms.IMRPhenomD import gen_IMRPhenomD_polar
from ripple.waveforms.IMRPhenomD import gen_IMRPhenomD
from jimgw.PE.detector_preset import * 
from jimgw.PE.heterodyneLikelihood import make_heterodyne_likelihood_mutliple_detector
from jimgw.PE.detector_projection import make_detector_response

psd_func_dict = {
    'H1': lalsim.SimNoisePSDaLIGOZeroDetHighPower,
    'L1': lalsim.SimNoisePSDaLIGOZeroDetHighPower,
    'V1': lalsim.SimNoisePSDAdvVirgo,
}
ifos = list(psd_func_dict.keys())


hp, hc = simulate_fd_sine_gaussian_waveform(A, t0, f0, tau, H1_d.times, f_min, H1_d.df)

H1 = get_H1()
H1_response = make_detector_response(H1[0], H1[1])   #detector_response(f, hp, hc, ra, dec, gmst, psi), strain in freq domain
L1 = get_L1()
L1_response = make_detector_response(L1[0], L1[1])   

gps = lal.LIGOTimeGPS(t0)
gmst_rad = lal.GreenwichMeanSiderealTime(gps)

#f_list = freqs[freqs>fmin]

L1_signal = L1_response(H1_d.freqs, hp, hc, ra, dec, gmst_rad, psi)   
H1_signal = H1_response(H1_d.freqs, hp, hc, ra, dec, gmst_rad, psi)  

H1_noise = 0
L1_noise = 0

#H1_noise = noise_fd_dict['H1'][freqs>fmin]
H1_data = H1_noise + H1_signal

#L1_noise = noise_fd_dict['L1'][freqs>fmin]
L1_data = L1_noise + L1_signal

response = {'H1': H1_response, 'L1': L1_response}


In [105]:
L1_response(H1_d.freqs, hp, hc, 2, 11, gmst_rad, psi)

Array([-1.59374413e+01+0.00000000e+00j, -1.71760402e+01+3.30585983e+01j,
       -2.22819160e+01+1.04653927e+02j, ...,
       -5.92151384e-04+2.52119695e-05j, -5.89091466e-04+6.51550868e-05j,
       -5.83343078e-04+1.04800473e-04j], dtype=complex128)

In [62]:
def project_to_detector(detector, hp, hc, ra, dec, gmst_rad, psi):                   #unfix the frequency array?
    """Compute the response of the detector to incoming strain """
    return response[detector](H1_d.freqs, hp, hc, ra, dec, gmst_rad, psi)

In [63]:
true_params = dict(
        A=10.,
        t0=0.5,
        f0=4.,
        tau=2./f0,
        ra=1.5,
        dec=0.5,
        psi=1.0)

detector = 'H1'
project_to_detector(detector, hp, hc, true_params['ra'], true_params['dec'], gmst_rad,  true_params['psi'] )

Array([4.22691916e+01+0.00000000e+00j, 5.08978328e+01+1.61448082e+01j,
       9.29365941e+01+5.19370812e+01j, ...,
       1.30238290e-03+8.80194250e-04j, 1.27418124e-03+9.20527249e-04j,
       1.24473685e-03+9.59961870e-04j], dtype=complex128)

In [171]:
true_params = dict(
        A=10.,
        t0=0.5,
        f0=4.,
        tau=2./f0,
        ra=1.5,
        dec=0.5,
        psi=1.0)

# likelihood function

class LogL(object):
    
    def __init__(self, true_gw_params):
        
        self.true_gw_params = true_gw_params
        self.detectors = {'H1': H1, 'L1': L1}
        
        gps_time = true_gw_params['t0']     #time of coalescence
        gps = lal.LIGOTimeGPS(gps_time)
        self.gmst_rad = lal.GreenwichMeanSiderealTime(gps)
        self.A = true_gw_params['A']
        self.t0 = true_gw_params['t0']
        self.f0 = true_gw_params['f0']
        self.tau = true_gw_params['tau']
        self.ra=true_gw_params['ra']
        self.dec = true_gw_params['dec']
        self.psi= true_gw_params['psi']

        self.f_min = 0.
        self.times = H1_d.times
        self.df = H1_d.df
   
        # simulate data
        #T = 1 # duration of analysis segment
        #srate = 20
        #df = 1/T
        #freqs = jnp.arange(1+srate*T, dtype=jnp.float32)*df
        #f0 = 100
        #Nbins = 1
        #i0 =int(f0/df)
        #self.freqs = jnp.atleast_2d(freqs[i0])

        #self.beta = 1/T

        #hp=true_gw_params['A']*jnp.cos(true_gw_params['phi'])     #plus polarisation
        #hc=true_gw_params['A']*jnp.sin(true_gw_params['phi'])    #cross polarisation
        
        self.hp, self.hc = simulate_fd_sine_gaussian_waveform(self.A, self.t0, self.f0, self.tau, H1_d.times, self.f_min, H1_d.df)
        self.data = self.simulate_response(self.hp, self.hc, self.ra, self.dec, self.psi)
        
        #self.bounds = dict(
        #    A = [self.true_params['A']*jnp.exp(-1), self.true_params['A']*jnp.exp(1)],
          #  phi = [-jnp.pi, jnp.pi],
         #  dec = [-jnp.pi/2, jnp.pi/2],
         #   ra = [0, jnp.pi*2],
         #   psi = [-jnp.pi/2, jnp.pi/2]
       # )

    def simulate_response(self, hp, hc, ra, dec, psi):
        r = {d: project_to_detector(d, hp, hc, ra, dec, self.gmst_rad, psi) for d in self.detectors.keys()}
        return r
    
    def __call__(self, params):
        hp, hc = simulate_fd_sine_gaussian_waveform(params['A'], self.t0, self.f0, self.tau, self.times, self.f_min, self.df)
        
        r = self.simulate_response(hp, hc, params['ra'], params['dec'], params['psi'])

        residuals = jnp.array([r[ifo] - self.data[ifo] for ifo in self.detectors.keys()])
        
        return -jnp.real(jnp.sum(residuals*jnp.conj(residuals),axis=(0,1)))/2

    @property
    def params(self):
        params = ['A','phi','dec','ra','psi']
        return params


    #@jax.jit
    def array_to_phys(self, x: Array) -> dict:
        
        p = dict()
        p['A'] = x[:,0] 
        p['ra'] = x[:,1]
        p['dec'] = x[:,2]
        p['psi'] = x[:,3]

        return p

In [172]:
#Neural network setup

from vi_routines import make_flow_model

@hk.without_apply_rng
@hk.transform
def sample_and_log_prob(prng_key: PRNGKey, n: int) -> Tuple[Any, Array]:

    model = make_flow_model(                          #this is the flow distribution (a distrax object)
        event_shape=(n_params,),
        num_layers=flow_num_layers,
        hidden_sizes=[hidden_size] * mlp_num_layers,
        num_bins=num_bins
    )

    return model.sample_and_log_prob(seed=prng_key, sample_shape=(n,))
     # returns x (sample from the flow q), and model.log_prob(x) (array of log(q) of th sampled points)


def log_likelihood(x: Array) -> Array:      #transform to parameter space before c
    p = log_l.array_to_phys(x)
    print(p)
    return log_l(p)

def loss_fn(params: hk.Params, prng_key: PRNGKey, n: int) -> Array:       #computes reverse KL-divergence for the sample x_flow between the flow and gw loglikelihood.

    x_flow, log_q = sample_and_log_prob.apply(params, prng_key, n)           #gets sample from the flow and computes log_q for the sampled points.
    log_p = log_likelihood(x_flow)
                                           #gets gw loglikelihood for the sampled points (after transforming them into physical space..)
    loss = jnp.mean(log_q - log_p)
    return loss

@jax.jit
def update(
    params: hk.Params,
    prng_key: PRNGKey,
    opt_state: OptState,
) -> Tuple[hk.Params, OptState]:
    """Single SGD update step."""
    grads = jax.grad(loss_fn)(params, prng_key, Nsamps)
    updates, new_opt_state = optimiser.update(grads, opt_state)
    new_params = optax.apply_updates(params, updates)
    return new_params, new_opt_state

In [173]:
n_params = 4
flow_num_layers = 4
hidden_size = 16
mlp_num_layers = 2
num_bins = 4


# perform variational inference
epochs = 100
loss = dict(train=[], val=[])
Nsamps = 100

learning_rate = 0.01
optimiser = optax.adam(learning_rate)              #stochastic gradient descent 

prng_seq = hk.PRNGSequence(42)
key = next(prng_seq)
params = sample_and_log_prob.init(key, prng_key=key, n=Nsamps)
opt_state = optimiser.init(params)

from tqdm import tqdm, trange
   
ldict = dict(loss = 0, T=1)

log_l = LogL(true_params)         

with trange(epochs) as tepochs:
    for epoch in tepochs:
  
        #update NN params (stochastic gradient descent with Adam optimiser)
        prng_key = next(prng_seq)
        loss = loss_fn(params,  prng_key, Nsamps)
        ldict['loss'] = f'{loss:.2f}'
        tepochs.set_postfix(ldict, refresh=True)
        params, opt_state = update(params, prng_key, opt_state)        #take a step in direction of stepest descent (negative gradient)
            
            
        #print results every 50 iterations
        if epoch%50 == 0:
            print(f'Epoch {epoch}, loss {loss}')
            x_gen, log_prob_gen = sample_and_log_prob.apply(params, next(prng_seq), 10*Nsamps)
            x_gen = np.array(x_gen, copy=False)
            p_gen = np.vstack(list(log_l.array_to_phys(x_gen).values()))
            truths = [log_l.true_gw_params[n] for n in ['A','ra','dec','psi']]
            fig = corner.corner(p_gen.T, labels = log_l.gw_params, truths = truths)
            plt.show()

print("Done!")

x_gen, log_prob_gen = sample_and_log_prob.apply(params, next(prng_seq), 100*Nsamps)

x_gen = np.array(x_gen, copy=False)
p_gen = np.vstack(list(log_l.array_to_phys(x_gen).values()))
truths = [log_l.true_gw_params[n] for n in ['A','B','C']]
fig = corner.corner(p_gen.T, labels = log_l.gw_params, truths = truths)
plt.show()

  0%|          | 0/100 [00:00<?, ?it/s]

{'A': Array([-0.5431577 ,  0.96932533, -0.3475862 ,  0.71317518,  0.24289826,
        0.6124214 ,  0.03533857,  0.2956065 , -0.79037099,  0.26872634,
        0.97840075,  0.33450018,  0.33042742, -0.44370358,  0.56553786,
       -0.32010291, -0.93359144, -0.8533547 ,  0.75119936, -0.59289556,
        0.51314745, -0.8727492 ,  0.1009783 ,  0.65345818, -0.30324559,
        0.8770263 , -0.03618788,  0.84738053,  0.67809712,  0.94322702,
       -0.24082463, -0.85283436, -0.95810391,  0.30869714, -0.17361   ,
       -0.84709579,  0.79757859,  0.94255953,  0.4196401 , -0.6562728 ,
       -0.44330852,  0.21718657, -0.31422378, -0.88270261,  0.46518684,
       -0.97747418,  0.67863249,  0.86348478,  0.87965681, -0.06704527,
        0.70349583, -0.05951069,  0.02736778, -0.41523776,  0.83175709,
        0.77379272,  0.66452742,  0.34733266, -0.54080936, -0.81759559,
        0.73358162, -0.08136186,  0.55466134, -0.98024018,  0.70347278,
       -0.44012189,  0.74985507, -0.52917137,  0.33750467,

ValueError: Incompatible shapes for broadcasting: shapes=[(100,), (1, 2048)]

In [166]:
x, logq = sample_and_log_prob.apply(params, prng_key, 100)

p = log_l.array_to_phys(x)
log_l(p)

#log_p = log_likelihood(x)

{'A': Array([-0.5431577 ,  0.96932533, -0.3475862 ,  0.71317518,  0.24289826,
        0.6124214 ,  0.03533857,  0.2956065 , -0.79037099,  0.26872634,
        0.97840075,  0.33450018,  0.33042742, -0.44370358,  0.56553786,
       -0.32010291, -0.93359144, -0.8533547 ,  0.75119936, -0.59289556,
        0.51314745, -0.8727492 ,  0.1009783 ,  0.65345818, -0.30324559,
        0.8770263 , -0.03618788,  0.84738053,  0.67809712,  0.94322702,
       -0.24082463, -0.85283436, -0.95810391,  0.30869714, -0.17361   ,
       -0.84709579,  0.79757859,  0.94255953,  0.4196401 , -0.6562728 ,
       -0.44330852,  0.21718657, -0.31422378, -0.88270261,  0.46518684,
       -0.97747418,  0.67863249,  0.86348478,  0.87965681, -0.06704527,
        0.70349583, -0.05951069,  0.02736778, -0.41523776,  0.83175709,
        0.77379272,  0.66452742,  0.34733266, -0.54080936, -0.81759559,
        0.73358162, -0.08136186,  0.55466134, -0.98024018,  0.70347278,
       -0.44012189,  0.74985507, -0.52917137,  0.33750467,

TypeError: mul got incompatible shapes for broadcasting: (100,), (2048,).

In [157]:
len(logq)

100